In [1]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('all-MiniLM-L6-v2')

In [3]:
def cos_sim_test(s1, s2):
    emb1 = model.encode(s1)
    emb2 = model.encode(s2)
    cos_sim = float(util.cos_sim(emb1, emb2))
    return cos_sim

In [4]:
tests = [
    ["cat", "dog"], 
    ["cat", "Cat"],
    ["Hark upon the Gale", "Once upon a time"],
    ["Hark upon the Gale", "Professor Angela Gao"]
]

In [7]:
for pair in tests:
    print(f"{round(cos_sim_test(pair[0], pair[1]),5)}  |  {pair}")

TypeError: round() takes at most 2 arguments (3 given)